In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim
from spandex import TableLoader

loader = TableLoader()

### Run all models

In [2]:
sim.run(["build_networks"]) #initialize network accessibility engine
sim.run(["scheduled_development_events", "neighborhood_vars", #scheduled events and accessibility variables
         "rsh_simulate", "nrh_simulate", "nrh_simulate2",   #price models
         "jobs_transition", "elcm_simulate", "households_transition", "hlcm_simulate", #demand/location models
         "price_vars", "feasibility", "residential_developer", "non_residential_developer", #supply/proforma models
         "model_integration_indicators" #indicators for pasef/pecas
         ], years=[2012, 2015,])

Running model 'build_networks'
Time to execute model 'build_networks': 13.81s
Total time to execute: 13.81s
Running year 2012
Running model 'scheduled_development_events'
Time to execute model 'scheduled_development_events': 0.90s
Running model 'neighborhood_vars'
Computing accessibility variables
Computing households_1500m
Computing jobs_3000m
Computing jobs_1500m
Computing jobs_800m
Computing jobs_400m
Computing jobs1
Computing jobs2
Computing jobs3
Computing jobs4
Computing jobs5
Computing jobs6
Computing jobs7
Computing jobs8
Computing jobs9
Computing jobs10
Computing jobs11
Computing jobs12
Computing jobs13
Computing jobs14
Computing jobs15
Computing jobs16
Computing jobs17
Computing jobs18
Computing jobs19
Computing jobs20
Computing ave_parcel_size
Computing ave_income
Computing ave_age_of_head_1500m
Computing ave_cars_1500m
Computing ave_children_1500m
Computing ave_year_built_1500m
Computing ave_sqft_per_unit_1500m
Computing population_400m
Computing population_800m
Computing p

KeyError: 'segmented_locationchoice'

In [ ]:
# LUZ prices
bsim = sim.get_table('buildings').to_frame(columns = ['luz_id', 'note', 'res_price_per_sqft', 'nonres_rent_per_sqft', 'residential_units', 'non_residential_sqft'])
luz_res_price = bsim[bsim.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = bsim[bsim.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
bsim = bsim[bsim.note == 'simulated']
luz_simdu = bsim.groupby('luz_id').residential_units.sum()
luz_simnr = bsim.groupby('luz_id').non_residential_sqft.sum()
luz_df = pd.DataFrame({'du':luz_simdu, 'nrsf':luz_simnr, 'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
luz_df.to_csv(loader.get_path('out/luz_sim_indicators.csv'))

In [ ]:
# Summarize results at MSA level
b = sim.get_table('buildings').to_frame(columns = ['mgra_id', 'residential_units', 'non_residential_sqft', 'note'])
mgra_msa_xref = pd.read_csv(loader.get_path("xref//mgra_msa.csv"))
merged = pd.merge(b, mgra_msa_xref, left_on = 'mgra_id', right_on = 'mgra_id')
new_du_by_msa = merged[merged.note  == 'simulated'].groupby('msa_id').residential_units.sum()
new_nrsf_by_msa = merged[merged.note  == 'simulated'].groupby('msa_id').non_residential_sqft.sum()
proportion_du_by_msa = new_du_by_msa / new_du_by_msa.sum()
proportion_nrsf_by_msa = new_nrsf_by_msa / new_nrsf_by_msa.sum()

In [ ]:
print proportion_du_by_msa
print proportion_nrsf_by_msa 

### Predict prices and summarize/export at the LUZ level

In [ ]:
%%capture
## Accessibility engine
sim.run(["build_networks", "neighborhood_vars"])

# Predict prices
sim.run(["rsh_simulate", "nrh_simulate", "nrh_simulate2"])

##Export LUZ mean prices to csv for visual checking
buildings = sim.get_table('buildings').to_frame()
luz_res_price = buildings[buildings.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = buildings[buildings.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
df = pd.DataFrame({'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
df.fillna(0).to_csv(loader.get_path('out/sim/sd_prices.csv'))

# Summarize prices using network queries
sim.run(['price_vars'])

### Proforma

In [ ]:
# from urbansim.developer import sqftproforma
# # Step 1:  feasibility
sim.run(['feasibility'])
sim.run(["residential_developer"])
sim.run(["non_residential_developer"])